In [1]:
import os


from dotenv import load_dotenv
load_dotenv()

#from langchain_openai import AzureChatOpenAI
from langchain.chat_models import ChatOpenAI

In [3]:
from datetime import datetime, timedelta
from typing import List
import math
import faiss
import logging
logging.basicConfig(level=logging.ERROR)
from langchain.docstore import InMemoryDocstore
#from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings

In [4]:
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS

from langchain_experimental.generative_agents import GenerativeAgent, GenerativeAgentMemory


In [5]:
USER_NAME = "Deepak"

In [6]:
# llm = AzureChatOpenAI(deployment_name=os.getenv("AZURE_DEP_NAME"),
#                          openai_api_version=os.getenv("AZURE_VERSION"),
#                           openai_api_key=os.getenv("AZURE_KEY"),
#                            azure_endpoint=os.getenv("AZURE_ENDPOINT"),
#                      max_tokens=1500)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(temperature=.7, model_name="gpt-3.5-turbo", max_tokens=1500)


/home/oem/workspace/envs/langchain-env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
def relevance_score_fn(score:float) -> float:
    """Returns similarity score on scale [0,1]"""
    return 1.0 - score/ math.sqrt(2)

In [9]:
from dotenv import load_dotenv
load_dotenv()
# os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_ENDPOINT")
# os.environ["api_key"] = os.getenv("AZURE_KEY")

#from langchain_openai import AzureOpenAIEmbeddings



def create_new_memory_retriever():
    """ Create new vector store retriever unique to the agent """
    # embeddings_model = AzureOpenAIEmbeddings(
    #                 azure_deployment=os.getenv("AZURE_DEP_NAME"),
    #                 openai_api_version=os.getenv("AZURE_VERSION"),
    #                 openai_api_key = os.getenv("AZURE_KEY"),
    #                 azure_endpoint = os.getenv("AZURE_ENDPOINT")
    # )
    embeddings_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
    
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {}, relevance_score_fn=relevance_score_fn)
    return TimeWeightedVectorStoreRetriever(vectorstore=vectorstore, other_score_keys=["importance"], k=15)


#create_new_memory_retriever()


In [10]:

from dotenv import load_dotenv
load_dotenv()

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_ENDPOINT")

os.environ["api_key"] = os.getenv("AZURE_KEY")

print(os.getenv("AZURE_KEY"))

from langchain_openai import AzureOpenAIEmbeddings


embeddings_model = AzureOpenAIEmbeddings(
                    azure_deployment=os.getenv("AZURE_DEP_NAME"),
                    openai_api_version=os.getenv("AZURE_VERSION"),
                    openai_api_key = os.getenv("AZURE_KEY"),
                    azure_endpoint = os.getenv("AZURE_ENDPOINT"),
                    model = "gpt-35-turbo-16k"
    )





0b096d64907e4f788295bf459d4757fa


In [11]:
alexis_memory = GenerativeAgentMemory(llm=llm,
                                      memory_retriever=create_new_memory_retriever(),
                                      verbose=False,
                                      reflection_threshold=8)

/home/oem/workspace/envs/langchain-env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [12]:
alexis = GenerativeAgent(
    name="Alexis",
    age=30,
    traits = "curious, creative writer, world traveller",
    status= "exploring the intersection of technology and storytelling",
    memory_retriever = create_new_memory_retriever(),
    llm=llm,
    memory = alexis_memory,
)

In [13]:
print(alexis.get_summary())

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveller
Alexis is confident, driven, and ambitious. She is a natural leader who is highly organized and goal-oriented. She is also empathetic and caring, always willing to help others in need.


In [15]:
alexis_observations = [
    "Alexis recalls her morning walk in the park",
    "Alexis feels excited about the new book she started reading",
    "Alexis remembers her conversation with a close friend",
    "Alexis thinks about the painting she saw at the art gallery",
    "Alexis is planning to learn a new recipe for dinner",
    "Alexis is lookin forward to her weekend trip",
    "Alexis contemplates her goals for the month."
]

for observation in alexis_observations:
    alexis.memory.add_memory(observation)

print(alexis.get_summary(force_refresh=True))

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveller
Alexis is reflective, goal-oriented, enjoys art and nature, values friendships, looks forward to new experiences, and is open to learning new things.


In [16]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """ Help the notebook user interact with the agent """
    new_message = "{0} says {1}".format(USER_NAME, message)
    return agent.generate_dialogue_response(new_message)[1]
    

In [17]:
interview_agent(alexis, "What do you like to do?")

'Alexis said "I enjoy writing, traveling, exploring art galleries, and spending time in nature. How about you, Deepak? What do you like to do?"'

In [18]:
alexis_observations_day = [
    "Alexis starts her day with a refreshing yoga session.",
    "Alexis spends her time writing in her journal.",
    "Alexis experiments with a new recipe she found online."
]

for observation in alexis_observations_day:
    alexis.memory.add_memory(observation)

In [25]:
from termcolor import colored
for i, obeservation in enumerate(alexis_observations_day):
    _, reaction = alexis.generate_reaction(obeservation)
    print(colored(obeservation, "green"), reaction)
    if ((i+1) % len(alexis_observations_day)) == 0:
        print("*" * 40)
        print(colored("After these observations, Alexis's Summary is :\n {0}".format(alexis.get_summary(force_refresh=True)), "blue"))
        print("*" * 40)

Alexis starts her day with a refreshing yoga session. Alexis smiles and feels refreshed after her yoga session, ready to start her day.
Alexis spends her time writing in her journal. Alexis feels inspired and reflective as she writes in her journal.
Alexis experiments with a new recipe she found online. Alexis feels excited to try out the new recipe she found online.
****************************************
After these pbservations, Alexis's Summary is :
 Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveller
Alexis is a reflective and creative individual who enjoys writing in her journal, starting her day with yoga to feel refreshed, and trying out new recipes. She values experiences like spending time in nature, visiting art galleries, and going on weekend trips. Alexis is goal-oriented, contemplates her goals for the month, and enjoys engaging in meaningful conversations with others.
****************************************


In [27]:
jordan = GenerativeAgent(
    name="Jordan",
    age=28,
    traits = "tech enthuist, avid gamer, foodie",
    status= "navigating the world of tech startups",
    memory_retriever = create_new_memory_retriever(),
    llm=llm,
    memory = alexis_memory,
)

In [28]:
jordan_observations = [
    "Jordan finished challenging coding project last night",
    "Jordan won local gaming tournament over the weekend",
    "Jordan tried new south indian restaurant and loved it"
    
]

for observation in jordan_observations:
    jordan.memory.add_memory(observation)

print(jordan.get_summary(force_refresh=True))

Name: Jordan (age: 28)
Innate traits: tech enthuist, avid gamer, foodie
Jordan is a skilled gamer and coder, enjoys exploring new restaurants, and is actively involved in various hobbies and activities. Alexis is reflective, enjoys writing and yoga, appreciates art and nature, and is focused on setting and achieving goals.


In [29]:
def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """ Runs conversation between agents """
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    max_turns = 3
    turn = 0
    while turn<=max_turns:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(observation)
            print(observation)
            # observation = "{0} said {1}".format(agent.name, reaction)
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turn +=1

In [31]:
agents = [alexis, jordan]
ini_conversation = "Alexis said: Hey Jordan, I have been exploring how technology influence creativity lately. Since you are into tech, I was wondering if you have seen something that can help me to undersatand."
run_conversation(agents, ini_conversation)

Jordan said "I'd love to discuss that with you! Let's set up a time to chat about it."
Alexis said "That sounds great! I'll message you to coordinate a time. Looking forward to our discussion! Have a wonderful day!"
Jordan said "Great! Looking forward to it. Have a wonderful day!"
Alexis said "It was great chatting with you, Jordan! Have a wonderful day!"
Jordan said "It was great chatting with you, Alexis! Have a wonderful day!"


In [32]:
interview_agent(alexis, "What was your conversation with Jordan?")

'Alexis said "I had a great discussion with Jordan about how technology influences creativity. We\'re planning to dive deeper into the topic soon. How about you, Deepak? What have you been up to lately?"'

In [33]:
interview_agent(jordan, "What was your conversation with Alexis?")

'Jordan said "It was great chatting with Alexis! Have a wonderful day!"'

In [35]:
def run_competitive_trivia(agents: List[GenerativeAgent], questions: list[str]) -> None:
    for question in questions:
        print("Trivia Question :: {0}".format(question))
        for agent in agents:
            response = agent.generate_dialogue_response(question)[1]
            print("{0}'s Answer:: {1}".format(agent.name, response))


agents = [alexis, jordan]
questions = [
    "Who is father of AI?",
    "What is the capital of India?",
    "Where is Burj Khalifa?"
]
run_competitive_trivia(agents, questions)

Trivia Question :: Who is father of AI?
Alexis's Answer:: Alexis said "I'm not sure who the Father of AI is, but I'm always curious to learn more about the pioneers in the field. Do you have any insights on this topic?"
Jordan's Answer:: Jordan said "I'm not sure who the Father of AI is, but I'm always curious to learn more about the pioneers in the field. Do you have any insights on this topic?"
Trivia Question :: What is the capital of India?
Alexis's Answer:: Alexis said "I believe the capital of India is New Delhi. It's a vibrant city with a rich history and culture."
Jordan's Answer:: Jordan said "I believe the capital of India is New Delhi. It's a vibrant city with a rich history and culture."
Trivia Question :: Where is Burj Khalifa?
Alexis's Answer:: Alexis said "I believe the Burj Khalifa is located in Dubai, United Arab Emirates. It's known for being the tallest building in the world and a popular tourist attraction."
Jordan's Answer:: Jordan said "I believe the Burj Khalifa 

In [37]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    )-> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = "{0}: ".format(self.name)
        self.reset()
    def reset(self) -> str:
        self.message_history = ["Here is the conversation so far"]

    def send(self) -> str:
        """
        Applied the chatmodel to the message history
        and return the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content
    def receive(self, name: str, message: str) -> None:
        """
        Concatenate {message} spoken by {name} into message history
        """
        self.message_history.append("{0}: {1}".format(name, message))
        



In [ ]:
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function = Callable[[init, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        